In [ ]:
import pandas as pd
import vaex
import os
import matplotlib.pyplot as plt
import datetime
from plotnine import *
import re

In [ ]:
path = r"F:\myresearch\TraumaHba1\data"
os.chdir(path)
sheets = os.listdir(path)
sheets

In [ ]:
"""
(
    ggplot(trauma[trauma["DAYS_SURV"].notna()], aes(x='DAYS_SURV'))
    + geom_histogram()
)
"""

In [ ]:
patients = pd.read_csv('trauma_addmission.csv')

In [ ]:
patients = patients[patients.columns[[3,4,18,22,28,29,27]]]

In [ ]:
patients.shape

In [ ]:
patients.HADM_ID.drop_duplicates().shape

In [ ]:
procedure_mv = pd.read_csv("trauma_procedure_mv.csv")

In [ ]:
## 有创通气
procedure_mv[(procedure_mv["ITEMID"] == 225792)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
#无创通气
procedure_mv[(procedure_mv["ITEMID"] == 225794)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
#插管
procedure_mv[(procedure_mv["ITEMID"] == 224385)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
#拔管
procedure_mv[(procedure_mv["ITEMID"] == 227194)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
#气管切开
procedure_mv[(procedure_mv["ITEMID"] == 225448)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
#非计划性拔管
procedure_mv[(procedure_mv["ITEMID"] == 225468)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
procedure_icd = pd.read_csv("trauma_procedure_icd.csv")

In [ ]:
procedure_icd.info()

In [ ]:
## 有创通气大于96小时
procedure_icd[(procedure_icd["ICD9_CODE"] == 9672)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
## 有创通气小于96小时
procedure_icd[(procedure_icd["ICD9_CODE"] == 9671)]["SUBJECT_ID"].drop_duplicates().shape

In [ ]:
## 无创通气
procedure_icd[(procedure_icd["ICD9_CODE"] == 9390)]["SUBJECT_ID"].drop_duplicates().shape

#### 提取机械通气病人号，合并入patients表格

In [ ]:
ventilation_icd9 = [9672,9671,9390,8669]
ventilation_icd_subjectid = procedure_icd[(procedure_icd["ICD9_CODE"].isin(ventilation_icd9))]["HADM_ID"].drop_duplicates()

ventilator_id = [225792,225794,227194,225448]
ventilation_mv_subjectid = procedure_mv[(procedure_mv["ITEMID"].isin(ventilator_id))]["HADM_ID"].drop_duplicates()

ventilation_subjectid = pd.concat([ventilation_icd_subjectid,ventilation_mv_subjectid]).drop_duplicates()

ventilation = pd.DataFrame({"HADM_ID":ventilation_subjectid,"ventilation":1})

patients = pd.merge(patients,ventilation,on="HADM_ID",how="left")

#### 提取crrt病历号，合并入patients

In [ ]:
crrt_mv_id = [225441,252802,225399,225400]
crrt_mv_subjectid = procedure_mv[(procedure_mv["ITEMID"].isin(crrt_mv_id))]["HADM_ID"].drop_duplicates()

crrt_icd_id = [3995]
crrt_icd_subjectid = procedure_icd[(procedure_icd["ICD9_CODE"].isin(crrt_icd_id))]["HADM_ID"].drop_duplicates()

crrt_subjectid = pd.concat([crrt_icd_subjectid,crrt_mv_subjectid]).drop_duplicates()

crrt = pd.DataFrame({"HADM_ID":crrt_subjectid,"crrt":1})

patients = pd.merge(patients,crrt,on="HADM_ID",how="left")

In [ ]:
chartevents = vaex.open("trauma_chartevents.hdf5")

##### 提取收缩压并合并

In [ ]:
bps_chart_id = [220179,220050,224167,227243,51,455,225309,6701,442]

bps_chart = chartevents[chartevents["ITEMID"].isin(bps_chart_id)]

bps_chart_df = bps_chart.to_pandas_df()

# bps_chart_df.info()

bps_chart_df[["CHARTTIME","STORETIME"]] = bps_chart_df[["CHARTTIME","STORETIME"]].apply(lambda x:pd.to_datetime(x))
## 提取收缩压的首次测量值
bps_first_time = bps_chart_df.groupby("HADM_ID").agg({"CHARTTIME":"min"})

bps=pd.merge(bps_first_time,bps_chart_df,on="CHARTTIME",how="left")[["HADM_ID","VALUENUM"]]

bps.drop_duplicates(subset=["HADM_ID"],inplace=True)

patients = pd.merge(patients,bps,on="HADM_ID",how="left")

patients.rename(columns={"VALUENUM":"bps"},inplace=True)

In [ ]:
bps.drop_duplicates(subset=["HADM_ID"],inplace=True)

In [ ]:
bpd_chart_id = [8368,8441,220051,220180,225310,8555,8440,224643,227242]

bpd_chart = chartevents[chartevents["ITEMID"].isin(bpd_chart_id)]

bpd_chart_df = bpd_chart.to_pandas_df()

# bpd_chart_df.info()

bpd_chart_df[["CHARTTIME","STORETIME"]] = bpd_chart_df[["CHARTTIME","STORETIME"]].apply(lambda x:pd.to_datetime(x))
## 提取收缩压的首次测量值
bpd_first_time = bpd_chart_df.groupby("SUBJECT_ID").agg({"CHARTTIME":"min"})

bpd=pd.merge(bpd_first_time,bpd_chart_df,on="CHARTTIME",how="left")[["HADM_ID","VALUENUM"]]

bpd.drop_duplicates(subset=["HADM_ID"],inplace=True)

patients = pd.merge(patients,bpd,on="HADM_ID",how="left")

patients.rename(columns={"VALUENUM":"bpd"},inplace=True)

In [ ]:
spo2_chart_id = [646,220277]

spo2_chart = chartevents[chartevents["ITEMID"].isin(spo2_chart_id)]

spo2_chart_df = spo2_chart.to_pandas_df()

# bpd_chart_df.info()

spo2_chart_df[["CHARTTIME","STORETIME"]] = spo2_chart_df[["CHARTTIME","STORETIME"]].apply(lambda x:pd.to_datetime(x))
## 提取收缩压的首次测量值
spo2_first_time = spo2_chart_df.groupby("SUBJECT_ID").agg({"CHARTTIME":"min"})

spo2=pd.merge(spo2_first_time,spo2_chart_df,on="CHARTTIME",how="left")[["HADM_ID","VALUENUM"]]

spo2.drop_duplicates(subset=["HADM_ID"],inplace=True)

patients = pd.merge(patients,spo2,on="HADM_ID",how="left")

patients.rename(columns={"VALUENUM":"spo2"},inplace=True)

In [ ]:
def add_vari_chartevents(chart_id_list,variable_name):
    
    global patients
     
    chart = chartevents[chartevents["ITEMID"].isin(chart_id_list)]
    
    chart_df = chart.to_pandas_df()

    # bpd_chart_df.info()

    chart_df[["CHARTTIME","STORETIME"]] = chart_df[["CHARTTIME","STORETIME"]].apply(lambda x:pd.to_datetime(x))
    ## 提取收缩压的首次测量值
    first_time = chart_df.groupby("HADM_ID").agg({"CHARTTIME":"min"})

    events=pd.merge(first_time,chart_df,on="CHARTTIME",how="left")[["HADM_ID","VALUENUM"]]

    events.drop_duplicates(subset=["HADM_ID"],inplace=True)
    
    patients = pd.merge(patients,events,on="HADM_ID",how="left")

    patients.rename(columns={"VALUENUM":variable_name},inplace=True)

In [ ]:
bps_chart_id = [220179,220050,224167,227243,51,455,225309,6701,442]
variable = "bps"

In [ ]:
add_vari_chartevents(bps_chart_id,variable)

In [ ]:
bpd_chart_id = [8368,8441,220051,220180,225310,8555,8440,224643,227242]
bpd = "bpd"

In [ ]:
add_vari_chartevents(bpd_chart_id,bpd)

In [ ]:
spo2_chart_id = [646,220277]
spo2="spo2"

In [ ]:
add_vari_chartevents(spo2_chart_id,spo2)

In [ ]:
patients

In [ ]:
labevents = pd.read_csv("trauma_labevents.csv",parse_dates=[4])

In [ ]:
"""
从chartevents 表中提取首次指标，加入总表
lab_id_list:列表，所需项目的ITEMID集合
variable_name:目标表格中新变量(新列)的名称
注意事项：1、总表需要定义成全局变量，并且需要根据具体情况改名
          2、仅适用于hdf5格式数据表
"""

def add_vari_labevents(lab_id_list,variable_name):
    
    global patients
    
    global labevents
     
    labevents = labevents[labevents["ITEMID"].isin(lab_id_list)]
    
    ## 提取收缩压的首次测量值
    first_time = labevents.groupby("HADM_ID").agg({"CHARTTIME":"min"})

    events=pd.merge(first_time,labevents,on="CHARTTIME",how="left")[["HADM_ID","VALUENUM"]]

    events.drop_duplicates(subset=["HADM_ID"],inplace=True)
    
    patients = pd.merge(patients,events,on="HADM_ID",how="left")

    patients.rename(columns={"VALUENUM":variable_name},inplace=True)

In [ ]:
lab_id_list = [58052]
variable_name="HbAc1"
add_vari_labevents(lab_id_list,variable_name)\

## 一个都没有

In [ ]:
chart_id_list = [861,1127,1542,220546]
variable_name = "wbc"
add_vari_chartevents(chart_id_list,variable_name)

In [ ]:
"""
向总表中加入变量列
"""
bps_chart_id = [220179,220050,224167,227243,51,455,225309,6701,442]
bpd_chart_id = [8368,8441,220051,220180,225310,8555,8440,224643,227242]
spo2_chart_id = [646,220277]
"""
构造字典，键为预定列名，值为ITEMID列表
"""
varid = {"bps":bps_chart_id,"bpd":bpd_chart_id,"spo2":spo2_chart_id}

for i in varid.items():
    #print(i[0])
    #print(i[1])
    ## 注意不同数据表调用不同函数
    add_vari_chartevents(i[1],i[0])

In [ ]:
patients

In [ ]:
(
    ggplot(patients[patients["wbc"].notna()], aes(x='wbc'))
    + geom_histogram()
)